# **🏠 네이버 부동산 크롤링**
> 안나오는 아파트들을 직접 찾아서 지번주소와 홈페이지를 불러옴  
> 직접 검색해도 안나오는 아파트는 결측치(공백)로 생성

## Contents
- Library Import
- Data Load
- Crawling of Naver real estate home page
- Extract attribute

## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [184]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

## 2. Data Load

In [185]:
# load
no_apt = pd.read_csv('../Data/검색안된아파트2.csv', encoding = 'cp949')

In [186]:
# 지번주소 없는거 다 빼기
n = no_apt[~no_apt['지번주소'].isna()]

- 생성해뒀던 파일에 중복값이 있는 경우가 존재했기 때문에 중복을 제거하고 진행

In [187]:
# 완전히 겹치는 행 제외시키기
n2 = n.drop_duplicates(['동아파트명', '지번주소', '페이지'])

In [188]:
# 지번주소에 있는 서울시 서울특별시로 변경
n2['지번주소'] = n2['지번주소'].str.replace('서울시', '서울특별시')

## 3. Crawling

In [192]:
ChromeDriverManager().install()
browser = webdriver.Chrome() # 우리가 컨트롤 할 수 있는 브라우저가 실행이된다.

In [193]:
# 열 이름을 리스트로 지정
column_names = ['동아파트명', '지번주소', '세대수', '저_최고층',
                 '총주차대수', '용적률', '건폐율', '건설사']

area_names = ['동아파트명', '지번주소', '공급전용면적', '방수욕실수',
              '해당면적세대수', '현관구조']


# 빈 데이터프레임 생성
apt_info_2nd = pd.DataFrame(columns=column_names)
apt_area_2nd = pd.DataFrame(columns=area_names)

In [287]:
# 데이터 확인
n2.values[:1330]

array([['개포동 개포자이', '서울특별시 강남구 개포동 12-2',
        'https://new.land.naver.com/complexes/8928?ms=37.496437,127.077115,17&a=APT:ABYG:JGC:PRE&e=RETAIL'],
       ['개포동 개포자이', '서울특별시 강남구 개포동 1283',
        'https://new.land.naver.com/complexes/145017?ms=37.478654,127.055232,17&a=APT:ABYG:JGC:PRE&e=RETAIL'],
       ['개포동 개포주공1단지', '서울특별시 강남구 개포동 660-1',
        'https://new.land.naver.com/complexes/134062?ms=37.479956,127.058861,17&a=APT:ABYG:JGC:PRE&e=RETAIL'],
       ...,
       ['동자동 센트레빌아스테리움서울', '서울특별시 용산구 동자동 45',
        'https://new.land.naver.com/complexes/101587?ms=37.551277,126.973633,17&a=APT:ABYG:JGC:PRE&e=RETAIL'],
       ['문배동 CJ나인파크', '서울특별시 용산구 문배동 24-6',
        'https://new.land.naver.com/complexes/19195?ms=37.536197,126.968816,17&a=APT:ABYG:JGC:PRE&e=RETAIL'],
       ['문배동 G&P용산파크', '서울특별시 용산구 문배동 11-34',
        'https://new.land.naver.com/complexes/122688?ms=37.537827,126.969471,17&a=APT:ABYG:JGC:PRE&e=RETAIL']],
      dtype=object)

In [302]:
# 재건축인 경우에 대한 번호
n2.reset_index().drop(['index'], axis = 1).loc[[55, 998, 1103, 1104, 1105, 1115, 1116], :]

,동아파트명,지번주소,페이지
55,대치동 쌍용대치,서울특별시 강남구 대치동 66,https://new.land.naver.com/complexes/185?ms=37...
998,반포동 에이아이디차관주택,서울특별시 서초구 반포동 1124,https://new.land.naver.com/complexes/453?ms=37...
1103,잠원동 신반포7,서울특별시 서초구 잠원동 65-32,https://new.land.naver.com/complexes/535?ms=37...
1104,잠원동 신반포8,서울특별시 서초구 잠원동 60-3,https://new.land.naver.com/complexes/536?ms=37...
1105,잠원동 신반포9,서울특별시 서초구 잠원동 60-6,https://new.land.naver.com/complexes/537?ms=37...
1115,성수동2가 청구강변1,서울특별시 성동구 성수동2가 712,https://new.land.naver.com/complexes/559?ms=37...
1116,성수동2가 청구강변1,서울특별시 성동구 성수동2가 745,https://new.land.naver.com/complexes/15648?ms=...


- 그대로 크롤링을 돌리게 되면 재건축 정보가 들어있기 때문에 XPATH가 달라지는 경우를 확인할 수 있음
- 따라서 따로 저장해서 불러오는 과정을 거쳐야 함

In [285]:
for i, [dong, bunji, homepage] in enumerate(n2.values[1330:1601]):

    # 창 불러오기
    browser.get(homepage)
    time.sleep(0.8)

    # 단지정보 클릭
    browser.find_element(By.CLASS_NAME, 'complex_link').click()
    time.sleep(1.5)

    # 세대수
    a1 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td[1]').text

    # 저/최고층
    a2 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td[2]').text
 

    # 총 주차대수
    a3 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[2]/td[2]').text

    # 용적률
    a4 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[3]/td[1]').text

    # 건폐율
    a5 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[3]/td[2]').text

    # 건설사
    a6 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[4]/td[1]').text

    # 데이터프레임에 집어넣기
    apt_info_2nd.loc[len(apt_info_2nd), :] = [dong, bunji, a1, a2, a3, a4, a5, a6]

    try:
        i = 0
        while True:
            
            # 단지내 면적별 정보 클릭
            browser.find_element(By.XPATH, f'//*[@id="tab{i}"]').click()

            # 공급/전용
            b1 = browser.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[1]').text

            # 방수/욕실수
            b2 = browser.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[2]').text
                    
            # 해당면적 세대수
            b3 = browser.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[3]').text

            # 현관구조
            b4 = browser.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[4]').text

            # 데이터에 집어넣기
            apt_area_2nd.loc[len(apt_area_2nd), :] = [dong, bunji, b1, b2, b3, b4]

            i += 1

    except:
        print(f"{dong}의 정보 담기를 {i}번까지 완료하였습니다.")
        pass

문배동 리첸시아용산B의 정보 담기를 7번까지 완료하였습니다.
문배동 삼라마이다스빌2의 정보 담기를 7번까지 완료하였습니다.
문배동 아크로타워의 정보 담기를 2번까지 완료하였습니다.
문배동 용산KCC웰츠타워의 정보 담기를 4번까지 완료하였습니다.
문배동 용산큐브의 정보 담기를 1번까지 완료하였습니다.
문배동 이안용산1차의 정보 담기를 6번까지 완료하였습니다.
문배동 이안용산3차의 정보 담기를 7번까지 완료하였습니다.
문배동 지오베르크의 정보 담기를 6번까지 완료하였습니다.
용산동5가 용산파크타워의 정보 담기를 7번까지 완료하였습니다.
원효로1가 로얄카운티-3의 정보 담기를 5번까지 완료하였습니다.
원효로2가 원효의 정보 담기를 7번까지 완료하였습니다.
이촌동 한강동부의 정보 담기를 4번까지 완료하였습니다.
이촌동 한강맨숀의 정보 담기를 7번까지 완료하였습니다.
이태원동 용산동의 정보 담기를 4번까지 완료하였습니다.
이태원동 이태원의 정보 담기를 2번까지 완료하였습니다.
청파동3가 한울글로리A의 정보 담기를 3번까지 완료하였습니다.
한강로1가 용산파크자이의 정보 담기를 5번까지 완료하였습니다.
한강로2가 용산푸르지오써밋의 정보 담기를 6번까지 완료하였습니다.
한남동 한남동리첸시아의 정보 담기를 7번까지 완료하였습니다.
수색동 DMC자이1단지의 정보 담기를 7번까지 완료하였습니다.
한강로2가 래미안용산더센트럴의 정보 담기를 6번까지 완료하였습니다.
한강로2가 벽산메가트리움의 정보 담기를 7번까지 완료하였습니다.
한강로2가 아스테리움용산의 정보 담기를 6번까지 완료하였습니다.
한강로2가 한강로대우아이빌의 정보 담기를 6번까지 완료하였습니다.
한강로3가 시티파크의 정보 담기를 6번까지 완료하였습니다.
한강로3가 시티파크의 정보 담기를 6번까지 완료하였습니다.
한강로3가 용산시티파크의 정보 담기를 6번까지 완료하였습니다.
한강로3가 용산시티파크의 정보 담기를 6번까지 완료하였습니다.
한강로3가 한강대우트럼프월드3의 정보 담기를 7번까지 완료하였습니다.
한강로3가 한강자이

In [294]:
# 저장
apt_info_2nd.to_csv('../Data/아파트정보_2nd.csv', index = False, encoding = 'cp949')
apt_area_2nd.to_csv('../Data/아파트면적정보_2nd.csv', index = False, encoding = 'cp949')

## 4. 파일 속성 뽑아내기

In [410]:
# load
apt_info = pd.read_csv('../Data/아파트정보.csv')
apt_area = pd.read_csv('../Data/아파트면적정보.csv')

apt_info2 = pd.read_csv('../Data/아파트정보2.csv')
apt_area2 = pd.read_csv('../Data/아파트면적정보2.csv')

In [411]:
# 두 컴퓨터에서 진행한 값 결합
ai = pd.concat([apt_info, apt_info2])
aa = pd.concat([apt_area, apt_area2]).reset_index().drop(['index'], axis = 1)

In [412]:
# 검색해서 안나온 값 없애기
first_ai = ai[~ai['세대수'].isna()].reset_index().drop(['index'], axis = 1)

In [413]:
# 세대수와 총 동수 구하기
first_ai['세대'] = first_ai['세대수'].apply(lambda x: x.split('세대(')[0]).astype(int)
first_ai['총동수'] = first_ai['세대수'].apply(lambda x: x.split('총')[1].split('개동')[0]).astype(int)

# '저_최고층' 열을 가지고 각 행에 대해 split 및 숫자 추출 수행
first_ai[['저층기준', '최고층']] = first_ai['저_최고층'].str.extract('(\d+)층/(\d+)층').astype(int)

# '총주차대수' 열을 가지고 각 행에 대해 숫자 및 소수점 추출 수행
result = first_ai['총주차대수'].str.extract('(\d+)대\(세대당 (\d+(\.\d+)?)대\)')
first_ai[['총주차면적개수', '세대당면적개수']] = result[[0, 1]].astype(float)

# '용적률' 열에서 '%' 제거
first_ai['용적비율'] = first_ai['용적률'].replace('-', np.nan).str.rstrip('%').astype(float)
first_ai['건폐비율'] = first_ai['건폐율'].replace('-', np.nan).str.rstrip('%').astype(float)

# 결측치 표현힉;
first_ai['건설사'] = first_ai['건설사'].replace('-', np.nan)

In [414]:
# str변수 완전 제거
apartment_infomation = first_ai.drop(['세대수', '저_최고층', '총주차대수', '용적률', '건폐율'], axis = 1)

In [415]:
# '공급전용면적' 열을 가지고 각 행에 대해 숫자 및 소수점 추출 수행
result = aa['공급전용면적'].str.extract('공급/전용 (\d+\.\d+)㎡/(\d+\.\d+)㎡\(전용률 (\d+)%\)')
aa[['공급면적', '전용면적', '전용율']] = result[[0, 1, 2]].astype(float)

# '방수욕실수' 열을 가지고 각 행에 대해 숫자 추출 수행
result = aa['방수욕실수'].str.extract('방수/욕실수 (\d+)개/(\d+)개')
aa[['방개수', '욕실개수']] = result.apply(lambda x: pd.to_numeric(x, errors='coerce')).astype('Int64')

# '해당면적세대수' 열을 가지고 각 행에 대해 숫자 추출 수행
result = aa['해당면적세대수'].str.extract('해당면적 세대수 (\d+)세대')
aa['면적당세대수'] = result[0].astype(int)

# '현관구조' 열을 가지고 각 행에 대해 '계단식' 추출 수행
result = aa['현관구조'].str.extract('현관구조 (\w+)')
aa['현관구조식'] = result[0]

In [416]:
# 전용면적 및 공급면적에 따른 정보 반영하기
apart_area_infomation = aa.drop(['공급전용면적', '방수욕실수', '해당면적세대수', '현관구조'], axis = 1)

In [417]:
# 두번째 기준
ai2 = pd.read_csv('../Data/아파트정보_2nd_comp.csv', encoding = 'cp949')
aa2 = pd.read_csv('../Data/아파트면적정보_2nd_comp.csv', encoding = 'cp949')

ai_dy = pd.read_csv('../Data/아파트정보_2nd_cp949.csv', encoding = 'cp949')
aa_dy = pd.read_csv('../Data/아파트면적정보_2nd_cp949_final_modified.csv', encoding = 'cp949')

second_ai = pd.concat([ai2, ai_dy], axis = 0)
second_aa = pd.concat([aa2, aa_dy], axis = 0)

In [418]:
# 세대수와 총 동수 구하기
second_ai['세대'] = second_ai['세대수'].apply(lambda x: x.split('세대(')[0]).astype(int)
second_ai['총동수'] = second_ai['세대수'].apply(lambda x: x.split('총')[1].split('개동')[0]).astype(int)

# '저_최고층' 열을 가지고 각 행에 대해 split 및 숫자 추출 수행
second_ai[['저층기준', '최고층']] = second_ai['저_최고층'].str.extract('(\d+)층/(\d+)층').astype(int)

# '총주차대수' 열을 가지고 각 행에 대해 숫자 및 소수점 추출 수행
result = second_ai['총주차대수'].str.extract('(\d+)대\(세대당 (\d+(\.\d+)?)대\)')
second_ai[['총주차면적개수', '세대당면적개수']] = result[[0, 1]].astype(float)

# '용적률' 열에서 '%' 제거
second_ai['용적비율'] = second_ai['용적률'].replace('-', np.nan).str.rstrip('%').astype(float)
second_ai['건폐비율'] = second_ai['건폐율'].replace('-', np.nan).str.rstrip('%').astype(float)

# 결측치 표현힉;
second_ai['건설사'] = second_ai['건설사'].replace('-', np.nan)

In [419]:
# str변수 완전 제거
apartment_infomation_2nd = second_ai.drop(['세대수', '저_최고층', '총주차대수', '용적률', '건폐율'], axis = 1)

In [421]:
# '공급전용면적' 열을 가지고 각 행에 대해 숫자 및 소수점 추출 수행
result = second_aa['공급전용면적'].str.extract('공급/전용 (\d+\.\d+)㎡/(\d+\.\d+)㎡\(전용률 (\d+)%\)')
second_aa[['공급면적', '전용면적', '전용율']] = result[[0, 1, 2]].astype(float)

# '방수욕실수' 열을 가지고 각 행에 대해 숫자 추출 수행
result = second_aa['방수욕실수'].str.extract('방수/욕실수 (\d+)개/(\d+)개')
second_aa[['방개수', '욕실개수']] = result.apply(lambda x: pd.to_numeric(x, errors='coerce')).astype('Int64')

# '현관구조' 열을 가지고 각 행에 대해 '계단식' 추출 수행
result = second_aa['현관구조'].str.extract('현관구조 (\w+)')
second_aa['현관구조식'] = result[0]

# '해당면적세대수' 열을 가지고 각 행에 대해 숫자 추출 수행
result = second_aa['해당면적세대수'].str.extract('해당면적 세대수 (\d+)세대')
second_aa['면적당세대수'] = result.apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(int)

In [422]:
# 전용면적 및 공급면적에 따른 정보 반영하기
apart_area_infomation_2nd = second_aa.drop(['공급전용면적', '방수욕실수', '해당면적세대수', '현관구조'], axis = 1)

In [424]:
# 데이터 확인
display(apartment_infomation,
apart_area_infomation,
apartment_infomation_2nd,
apart_area_infomation_2nd)

,동아파트명,건설사,세대,총동수,저층기준,최고층,총주차면적개수,세대당면적개수,용적비율,건폐비율
0,개포동 개포6차우성,우성건설(주),270,8,5,5,270.0,1.00,106.0,21.0
1,개포동 개포우성3차,우성건설(주),405,5,15,15,450.0,1.11,179.0,15.0
2,개포동 개포주공6단지,대한주택공사,1060,9,13,15,300.0,0.28,NaN,NaN
3,논현동 마일스디오빌,(주)대우건설,260,1,15,15,200.0,0.76,599.0,54.0
4,논현동 아크로힐스논현,대림산업(주),368,4,16,30,645.0,1.75,299.0,24.0
...,...,...,...,...,...,...,...,...,...,...
5824,신내동 화성.두산,두산.화성건설,763,8,15,15,763.0,1.00,241.0,20.0
5825,장지동 송파파인타운7단지,극동건설(주),537,6,10,17,576.0,1.07,239.0,22.0
5826,장지동 송파파인타운9단지,(주)신일건업,796,7,8,17,822.0,1.03,234.0,22.0
5827,숭인동 종로센트레빌,동부건설㈜,416,6,6,12,514.0,1.23,187.0,22.0


,동아파트명,공급면적,전용면적,전용율,방개수,욕실개수,면적당세대수,현관구조식
0,개포동 개포6차우성,61.32,54.98,90.0,2,1,20,계단식
1,개포동 개포6차우성,73.72,67.28,91.0,3,1,70,계단식
2,개포동 개포6차우성,86.52,79.97,92.0,3,1,180,계단식
3,개포동 개포우성3차,110.60,104.43,94.0,3,1,135,계단식
4,개포동 개포우성3차,153.01,133.46,87.0,4,2,150,계단식
...,...,...,...,...,...,...,...,...
26528,신내동 신내우디안1단지,84.91,59.54,70.0,3,2,2,계단식
26529,신내동 신내우디안1단지,84.91,59.54,70.0,3,2,2,계단식
26530,신내동 신내우디안1단지,84.77,59.52,70.0,3,2,2,계단식
26531,신내동 신내우디안1단지,84.87,59.52,70.0,3,2,2,계단식


,동아파트명,지번주소,건설사,Unnamed: 8,Unnamed: 9,세대,총동수,저층기준,최고층,총주차면적개수,세대당면적개수,용적비율,건폐비율
0,개포동 개포자이,서울특별시 강남구 개포동 12-2,LG건설(주),NaN,NaN,212,4,20,22,502.0,2.36,250.0,18.0
1,개포동 개포자이,서울특별시 강남구 개포동 1283,자이에스앤디(주),NaN,NaN,28,1,8,10,38.0,1.35,199.0,57.0
2,개포동 개포주공1단지,서울특별시 강남구 개포동 660-1,"현대건설(주), 현대산업개발",NaN,NaN,6702,74,7,35,13154.0,1.96,249.0,19.0
3,논현동 동양파라곤,서울특별시 강남구 논현동 245,동양고속건설(주),NaN,NaN,203,4,9,16,572.0,2.81,264.0,22.0
4,대치동 대치삼성,서울특별시 강남구 대치동 1014-3,삼성물산 주식회사,NaN,NaN,960,14,7,25,1259.0,1.31,261.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,잠원동 신반포12,서울특별시 서초구 잠원동 50-5,한신공영㈜,NaN,NaN,324,3,12,12,312.0,0.96,NaN,NaN
686,잠원동 신반포17,서울특별시 서초구 잠원동 58-22,한신공영㈜,NaN,NaN,216,3,12,12,216.0,1.00,NaN,NaN
687,잠원동 신반포21,서울특별시 서초구 잠원동 59-10,한신공영㈜,NaN,NaN,108,2,8,10,116.0,1.07,170.0,19.0
688,잠원동 신반포22,서울특별시 서초구 잠원동 65-33,한신공영㈜,NaN,NaN,132,1,12,12,NaN,NaN,166.0,14.0


,동아파트명,지번주소,공급면적,전용면적,전용율,방개수,욕실개수,현관구조식,면적당세대수
0,개포동 개포자이,서울특별시 강남구 개포동 12-2,160.71,134.30,84.0,4,2,계단식,84
1,개포동 개포자이,서울특별시 강남구 개포동 12-2,183.14,153.80,84.0,4,2,계단식,44
2,개포동 개포자이,서울특별시 강남구 개포동 12-2,201.74,170.32,84.0,4,2,계단식,42
3,개포동 개포자이,서울특별시 강남구 개포동 12-2,204.52,168.42,82.0,4,2,계단식,42
4,개포동 개포자이,서울특별시 강남구 개포동 1283,71.89,50.86,71.0,2,1,계단식,18
...,...,...,...,...,...,...,...,...,...
3185,잠원동 신반포22,서울특별시 서초구 잠원동 65-33,128.87,118.89,92.0,4,2,계단식,8
3186,잠원동 신반포22,서울특별시 서초구 잠원동 65-33,128.88,118.90,92.0,4,2,계단식,1
3187,성수동2가 강변청구3,서울특별시 성동구 성수동2가 745,73.38,59.94,82.0,2,1,복도식,36
3188,성수동2가 강변청구3,서울특별시 성동구 성수동2가 745,103.91,84.87,82.0,3,2,계단식,192


In [1]:
# 추가적으로 생기는 열 삭제
tmp = apartment_infomation_2nd.drop(['Unnamed: 8', 'Unnamed: 9'], axis = 1)


In [456]:
# groupby로 합을 구해야할 열 계산
tmp2_1 = tmp[['동아파트명', '지번주소', '세대', '총동수', '총주차면적개수']].groupby(['동아파트명', '지번주소']).sum().reset_index().sort_values(['동아파트명', '지번주소'])


In [455]:
# 동아파트명과 지번주소가 겹치는 값을 제거 
tmp2 = tmp[['동아파트명', '지번주소', '건설사', '저층기준', '최고층', '용적비율', '건폐비율']].drop_duplicates(['동아파트명', '지번주소']).sort_values(['동아파트명', '지번주소'])


In [478]:
# 101동, 102동과 같이 나뉘어진 경우 값이 완전히 똑같기 때문에 값을 합쳐주어야됨
apartment_infomation_2nd = pd.merge(tmp2_1, tmp2, how = 'outer', on = ['동아파트명', '지번주소']).reset_index().drop(['index'], axis = 1)

In [479]:
# 새롭게 계산해야하는 세대당 면적개수
apartment_infomation_2nd['세대당면적개수'] = np.round(apartment_infomation_2nd['총주차면적개수']/apartment_infomation_2nd['세대'], 2)

In [489]:
# 면적 정보 또한 뽑아줘여함
a1 = apart_area_infomation_2nd[['동아파트명', '지번주소', '전용면적', '면적당세대수']].groupby(['동아파트명', '지번주소', '전용면적']).sum().reset_index().sort_values(['동아파트명', '지번주소', '전용면적'])

In [490]:
# 동아파트명, 지번주소, 전용면적으로 겹치는 값을 제거
a2 = apart_area_infomation_2nd.drop(['면적당세대수'], axis = 1).drop_duplicates(['동아파트명', '지번주소', '전용면적'])

In [503]:
# 값을 합쳐주면서 겹치는 값이 없게 만들어줌
a3 = pd.merge(a1, a2, how = 'right', on = ['동아파트명', '지번주소', '전용면적'])

In [512]:
# 저장
apartment_infomation.drop_duplicates(['동아파트명']).to_csv('../Data/소중한정보_인포.csv', index = False)
apart_area_infomation.drop_duplicates(['동아파트명', '전용면적']).to_csv('../Data/소중한정보_에어리어.csv', index = False)
apartment_infomation_2nd.to_csv('../Data/소중한정보_두번째_인포.csv', index = False)
a3.to_csv('../Data/소중한정보_두번째_에어리어.csv', index = False)